# LangGraph: Time Travel and State Editing

## Introduction
One of the most powerful features of LangGraph's persistence layer is the ability to revisit previous states, modify them, and continue execution from that point. This is often called **Time Travel**.

---

## 1. Setup
We use the same checkpointer setup as before.

In [ ]:
import sqlite3
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_openai import ChatOpenAI

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State):
    return {"messages": [ChatOpenAI(model="gpt-4o-mini").invoke(state["messages"])]}

conn = sqlite3.connect(":memory:", check_same_thread=False)
memory = SqliteSaver(conn)

builder = StateGraph(State)
builder.add_node("chatbot", chatbot)
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)
graph = builder.compile(checkpointer=memory)

## 2. Creating a History
We will run a few turns to generate checkpoints.

In [ ]:
config = {"configurable": {"thread_id": "time_travel_demo"}}

graph.invoke({"messages": [("user", "Hi, I am Rakesh.")]}, config)
graph.invoke({"messages": [("user", "What is my name?")]}, config)

for msg in graph.get_state(config).values["messages"]:
    msg.pretty_print()

## 3. Browsing History (Time Travel)
We can list all previous states (checkpoints) for this thread.

In [ ]:
all_states = [s for s in graph.get_state_history(config)]
print(f"Found {len(all_states)} historical states.")

# Look at an older state
old_state = all_states[-2]
print("--- OLD STATE MESSAGES ---")
for msg in old_state.values["messages"]:
    msg.pretty_print()

## 4. State Editing and Forking
We can overwrite the state to correct history or fork it into a new direction.

In [ ]:
# Let's change the name the user gave
from langchain_core.messages import HumanMessage

# update_state creates a NEW checkpoint
graph.update_state(
    config, 
    {"messages": [HumanMessage(content="Actually, my name is Bob.", id=all_states[-1].values['messages'][0].id)]},
    as_node="chatbot"
)

print("--- UPDATED STATE ---")
for msg in graph.get_state(config).values["messages"]:
    msg.pretty_print()

## Summary
1.  **get_state_history**: View the past.
2.  **update_state**: Change the present or the past.
3.  **Checkpointing**: Every step is saved, allowing you to fork by reverting to an old configuration and starting a new path.